<a href="https://colab.research.google.com/github/benjaminbrown038/Hugging-Face/blob/main/video_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging Face Video Classification

In [ ]:
%%capture
!pip3 install pytorchvideo transformers evaluate

In [ ]:
%%capture
from huggingface_hub import notebook_login, hf_hub_download
import tarfile
from transformers import VideoMAEImageProcessor, VideoMAEForVideoClassification, TrainingArguments, Trainer, pipeline
import pytorchvideo.data
from pytorchvideo.transforms import ApplyTransformToKey, Normalize, RandomShortSideScale, RemoveKey, ShortSideScale, UniformTemporalSubsample
from torchvision.transforms import Compose, Lambda, RandomCrop, RandomHorizontalFlip, Resize
import imageio
import numpy as np
from IPython.display import Image
import evaluate

In [ ]:
hf_dataset_identifier = "sayakpaul/ufc101-subset"
filename = "UFC101_subset.tar.gz"
file_path = hf_hub_download(repo_id=hf_dataset_identifier,filename=filename,repo_type="dataset")


In [ ]:
with tarfile.open(file_path) as t:
    t.extractall(".")

In [ ]:
class_labels = sorted({str(path).split("/")[2] for path in all_video_file_paths})
label2id = {label:i for i, label in enumerate(class_labels)}
id2label = {i: label for label, i in label2id.items()}

print(f"Unique classes: {list(label2id.keys())}.")


In [ ]:
model_ckpt = "MCG-NJU/videomae-base"
image_processor = VideoMAEImageProcessor.from_pretrained(model_ckpt)
model = VideoMAEForVideoClassification.from_pretrained(
    model_ckpt,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True
)

In [ ]:
mean = image_processor.image_mean
std = image_processor.image_std
if "shortest_edge" in image_processor.size:
    height = width = image_processor.size["shortest_edge"]
else:
    height = image_processor.size["height"]
    width = image_processor.size["width"]
resize_to = (height,width)
num_frames_to_sample = model.config.num_frames
sample_rate = 4 
fps = 30 
clip_duration = num_frames_to_sample * sample_rate / fps


In [ ]:
def

In [ ]:
def unnormalize_img():

In [ ]:
def create_gif():

In [ ]:
def display_gif():
  

In [ ]:
sample_video = next(iter(train_dataset))
video_tensor = sample_video["video"]
display_gif(video_tensor)

In [ ]:
model_name = model_ckpt.split("/")[-1]
new_model_name = f"{model_name}-finetuned-ucf101-subset"
num_epochs = 4
args = TrainingArguments(
    new_model_name,
    remove_unused_columns=,
    evaluation_strategy,
    save_strategy,
    learning_rate,
    per_device_train_batch_size,
    per_device_eval_batch_size,
    warmup_ratio,
    logging_steps,
    load_best_model_at_end,
    metric_for_best_model,
    push_to_hub,
    max_steps
)

In [ ]:
metric = evaluate.load()
def compute_metrics():
    predictions = np.argmax(eval_pred.predictions,axis=1)
    return metric.compute(predictions = predictions, references = eval_pred.label_ids)
    

In [ ]:
def collate_fn():
    pixel_values = torch.stack(
        [example["video"].permute(1,0,2,3) for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values,"labels":labels}

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset,
    eval_dataset,
    tokenizer,
    compute_metrics,
    data_collator)

In [ ]:
train_results = trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
video_cls = pipeline(model="my_awesome_video_cls_model")
video_cls("https://huggingface.co/datasets/sayakpaul/ucf101-subset/resolve/main/v_BasketballDunk_g14_c06.avi")


In [ ]:
def run_inference():
    permuted_sample_test_video = video.permute(1,0,2,3)
    inputs = {
        "pixel_values": permuted_sample_test_video.unsqueeze(0),
        "labels": torch.tensor(
            [sample_test_video["label"]]
        )
    }

    device = torch.device("cuda" if torch.cuda_is_available() else "cpu")
    inputs = {k: v.to(device) for k,v in inputs.items()}
    model = model.to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    return logits

In [ ]:
logits = run_inference(trained_model,sample_test_video["video"])

In [ ]:
predicted_class_idx = logits.argmax(-1).item()
print("predicted class:", model.config.id2label[predicted_class_idx])
